In [6]:
!pip install datasets
!pip install rouge-score
!pip install rouge

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [1]:
from datasets import load_dataset
import re
from nltk.tokenize import sent_tokenize
import nltk
import numpy as np
import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/kd1860/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
dataset = load_dataset('multi_news', split='train')

Found cached dataset multi_news (/home/kd1860/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)


In [3]:
from rouge import Rouge
rouge = Rouge()

In [4]:
def filter_sentences(doc):
    document = doc['document']
    splited_document = sent_tokenize(document)
    scores = []
    for idx in range(len(splited_document)):
        target = splited_document[idx]
        rest_doc = ' '.join(splited_document[:idx] + splited_document[idx+1:])
        # print(rest_doc)
        try:
            scores.append(rouge.get_scores(target,rest_doc)[0]['rouge-1']['f'])
        except:
            scores.append(0)

    rank = (-np.array(scores)).argsort()
    threshold = int(len(rank)*0.8)
    rank = rank[:threshold]
    rank.sort()
    filtered_doc = ' '.join([splited_document[i] for i in rank])
    doc['document'] = filtered_doc

In [7]:
small_dataset = dataset.select(list(range(10)))
small_dataset

Dataset({
    features: ['document', 'summary'],
    num_rows: 10
})

In [8]:
import time 
start  = time.time()
small_dataset.map(filter_sentences, num_proc = 48)
end = time.time()
print (end-start)

num_proc must be <= 10. Reducing num_proc to 10 for dataset of size 10.


#0:   0%|          | 0/1 [00:00<?, ?ex/s]

#1:   0%|          | 0/1 [00:00<?, ?ex/s]

#2:   0%|          | 0/1 [00:00<?, ?ex/s]

#3:   0%|          | 0/1 [00:00<?, ?ex/s]

...

#6:   0%|          | 0/1 [00:00<?, ?ex/s]

#7:   0%|          | 0/1 [00:00<?, ?ex/s]

#8:   0%|          | 0/1 [00:00<?, ?ex/s]

#9:   0%|          | 0/1 [00:00<?, ?ex/s]

12.500081300735474


In [9]:
small_dataset

Dataset({
    features: ['document', 'summary'],
    num_rows: 10
})